In [3]:
# things we need for NLP
import string
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
#from nltk.stem import WordNetLemmatizer
#lemmatizer = WordNetLemmatizer()
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
import json
intents_ = {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi there", "How are you", "Is anyone there?", "Hello", "Good day"],
         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
         "context": [""]
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
         "context": [""]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"],
         "context": [""]
        },
        {"tag": "noanswer",
         "patterns": [],
         "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
         "context": [""]
        },
        {"tag": "options",
         "patterns": ["How you could help me?", "What you can do?", "What help you provide?", "How you can be helpful?", "What support is offered"],
         "responses": ["I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies", "Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies"],
         "context": [""]
        },
        {"tag": "adverse_drug",
         "patterns": ["How to check Adverse drug reaction?", "Open adverse drugs module", "Give me a list of drugs causing adverse behavior", "List all drugs suitable for patient with adverse reaction", "Which drugs dont have adverse reaction?" ],
         "responses": ["Navigating to Adverse drug reaction module"],
         "context": [""]
        },
        {"tag": "blood_pressure",
         "patterns": ["Open blood pressure module", "Task related to blood pressure", "Blood pressure data entry", "I want to log blood pressure results", "Blood pressure data management" ],
         "responses": ["Navigating to Blood Pressure module"],
         "context": [""]
        },
        {"tag": "blood_pressure_search",
         "patterns": ["I want to search for blood pressure result history", "Blood pressure for patient", "Load patient blood pressure result", "Show blood pressure results for patient", "Find blood pressure results by ID" ],
         "responses": ["Please provide Patient ID", "Patient ID?"],
         "context": ["search_blood_pressure_by_patient_id"]
        },
        {"tag": "search_blood_pressure_by_patient_id",
         "patterns": [],
         "responses": ["Loading Blood pressure result for Patient"],
         "context": [""]
        },
        {"tag": "pharmacy_search",
         "patterns": ["Find me a pharmacy", "Find pharmacy", "List of pharmacies nearby", "Locate pharmacy", "Search pharmacy" ],
         "responses": ["Please provide pharmacy name"],
         "context": ["search_pharmacy_by_name"]
        },
        {"tag": "search_pharmacy_by_name",
         "patterns": [],
         "responses": ["Loading pharmacy details"],
         "context": [""]
        },
        {"tag": "hospital_search",
         "patterns": ["Lookup for hospital", "Searching for hospital to transfer patient", "I want to search hospital data", "Hospital lookup for patient", "Looking up hospital details" ],
         "responses": ["Please provide hospital name or location"],
         "context": ["search_hospital_by_params"]
        },
        {"tag": "search_hospital_by_params",
         "patterns": [],
         "responses": ["Please provide hospital type"],
         "context": ["search_hospital_by_type"]
        },
        {"tag": "search_hospital_by_type",
         "patterns": [],
         "responses": ["Loading hospital details"],
         "context": [""]
        }
   ]
}
#Convert dict to JSON
#intents = json.dumps(intents_)
#Writing JSON to a file
with open('intents.json', 'w') as f:
  json.dump(intents_, f)

In [0]:
# import our chat-bot intents file
#Reading JSON file 
with open('intents.json') as f:  
  intents = json.load(f)

In [0]:
#Helper function to clean sentances
def clean(sentence):
    words = nltk.word_tokenize(sentence)
    words = [stemmer.stem(word.lower()) for word in words]
    return words

In [0]:
#words: list of words in all documents
#classes: list of tags 
#documents: list of lists, each list has words for one doc and the tag of doc
words = []
classes = []
documents = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
      pattern_words=clean(pattern)
      words += pattern_words
      documents.append((pattern_words, intent['tag']))
      # add to our classes list
      if intent['tag'] not in classes:
          classes.append(intent['tag'])

# sort classes
classes = sorted(list(set(classes)))
words = sorted(list(set(words)))

In [8]:
print (len(documents), "documents" ,", documents[0]:",documents[0])
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

45 documents , documents[0]: (['hi', 'ther'], 'greeting')
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
83 unique stemmed words ["'s", ',', '?', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [0]:
'''prepare data for model, to train_x, train_y
   train_x: has list of lists. each list represent words of doc, list of zeros and ones, 
           if word in doc it will be one else it will be zero.
  train_y: has list of lists. each list represent tag of doc. list has all zeros expect one value will be one
'''
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/200





45/45 [==============================] - 8s 183ms/step - loss: 2.2999 - acc: 0.0667
Epoch 2/200
45/45 [==============================] - 0s 691us/step - loss: 2.1103 - acc: 0.2000
Epoch 3/200
45/45 [==============================] - 0s 673us/step - loss: 2.0439 - acc: 0.2667
Epoch 4/200
45/45 [==============================] - 0s 843us/step - loss: 1.8942 - acc: 0.4222
Epoch 5/200
45/45 [==============================] - 0s 641us/step - loss: 1.8466 - acc: 0.4000
Epoch 6/200
45/45 [==============================] - 0s 654us/step - loss: 1.7759 - acc: 0.4889
Epoch 7/200
45/45 [==============================] - 0s 769us/step - loss: 1.6500 - acc: 0.5111
Epoch 8/200
45/45 [==============================] - 0s 663us/step - loss: 1.4550 - acc: 0.5778
Epoch 9/2

In [0]:
# save model to file
pickle.dump(model, open("katana-assistant-model.pkl", "wb"))

In [0]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "katana-assistant-data.pkl", "wb" ) )

In [0]:

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
def chat():
  print("Start talking with the bot (type quit to stop)!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    # generate probabilities from the model
    input_data = pd.DataFrame([bow(inp, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    results_index = np.argmax(results)
    tag = classes[results_index]
    #return_list=[(classes[results_index], results[results_index])]
    if results[results_index] >= 0.7:
          for tg in intents["intents"]:
              if tg['tag'] == tag:
                  responses = tg['responses']
          print(random.choice(responses))
    else:
      print("I can't understand. Try again.")

In [0]:
chat()

Start talking with the bot (type quit to stop)!
You: Hi
Good to see you again
You: where can i find the pharmacy?
Please provide pharmacy name
You: bey
I can't understand. Try again.
You: bye
Bye! Come back again soon.
